# Como determinar a similaridade entre textos em Python

In [214]:
# bibliotecas para carregar as imagens
from IPython.display import Image
from IPython.core.display import HTML 
import pandas as pd
import numpy as np
import string

## Conceitos

### O que é similaridade de texto?

A similaridade de texto determina o quão "próximas" duas partes do texto estão, tanto em proximidade de superfície **similaridade lexical** e significado **similaridade semântica**.


Por exemplo, quão semelhantes são as frases “o gato comeu o rato” com “o rato comeu a comida do gato” apenas olhando para as palavras?

* Superficialmente, se você considerar apenas a similaridade no nível da palavra, essas duas frases parecem muito semelhantes, pois 3 das 4 palavras únicas são uma sobreposição exata. Normalmente não leva em consideração o significado real por trás das palavras ou a frase inteira no contexto.
* Em vez de fazer uma comparação palavra por palavra, também precisamos prestar atenção ao contexto para capturar mais da semântica. Para considerar a similaridade semântica, precisamos nos concentrar nos níveis de frase / parágrafo (ou nível de cadeia lexica), onde um trecho de texto é dividido em um grupo relevante de palavras relacionadas antes de calcular a similaridade. Sabemos que, embora as palavras se sobreponham significativamente, essas duas frases, na verdade, têm significados diferentes.

Existe uma estrutura de dependência em qualquer frase:
* rato é o objeto de comer no primeiro caso e comida é o objeto de comer no segundo caso

Uma vez que as diferenças na ordem das palavras frequentemente andam de mãos dadas com as diferenças de significado (compare o cachorro morde o homem com o homem morde o cachorro), gostaríamos que nossos encaixes de frases fossem sensíveis a essa variação.

### Distancia e Similaridade

Dado dois pontos em um plano cartesiano, para entender como esses dois pontos estão relacionados entre si podemos nos perguntar: 

* "Qual a que distância ou quão próximos estão esses dois pontos?"
* A resposta para "Quão distantes estão esses pontos?" é a distância deles!
* A resposta para "Quão próximos estão esses pontos?" é a sua semelhança! 

Além dessa distinção, similaridade, pode se referir a uma categoria maior de medidas de similaridade, enquanto a distância geralmente se refere a uma categoria menos abrangente que mede a diferença no espaço cartesiano.

Pode parecer redundante ou confuso usar os dois termos, mas em análise de texto esses conceitos são geralmente relacionados bem relacionados (ou seja, distância é meramente o oposto de similaridade e vice-versa). 

No mundo todo, é provável que você encontre os dois termos. Quando medimos por distância, os pontos mais próximos terão a menor distância, mas quando você está medindo por semelhança, os pontos mais próximos terão a maior semelhança. Existem muitas maneiras de calcular a distância entre dois pontos no espaço cartesiano, e diferentes medidas de distância são úteis para diferentes propósitos.

Neste documento abordaremos algumas formas de realizar estes calculos.

### Qual é a estrategia ?

A grande ideia é representar os textos como vetores comparando os textos e medindo a distância entre eles. Existem várias maneiras de computar as distancias que capturam a semântica de textos e vários algoritmos para capturar a estrutura de dependência de textos para enfocar nos significados dos textos.

* Word2Vec + Smooth Inverse Frequency + Cosine Similarity
* Different embeddings+ LDA + Jensen-Shannon distance 
* Different embeddings+ Word Mover Distance 
* Different embeddings+ Siamese Manhattan LSTM
* Fuzzy +  Levenshtein Distance 
* BERT embeddings + Cosine Similarity

A word embedding é uma das representações mais populares do vocabulário. 

É capaz de capturar o contexto de uma palavra em um textos, semelhança semântica e sintática, relação com outras palavras, etc. Abaixo alguns exemplos de modelos de word embedding:

* Word2Vec (Google)
* GloVe (Stanford)
* fastText (Facebook)
* Poincarré 
* Node2Vec embedding baseado em Random Walk e Grafos

## Exemplo do Estudo

Uma das coisas que mais serviu como base para outras obras de ficção – e mais recentemente até para a realidade – e que foi invenção de Asimov são as famosas Três Leis da Robótica. Esse conjunto de regras apareceu pela primeira vez em um conto chamado “Círculo Vicioso”, que aparece em “Eu, Robô”. Segundo a história, essas leis teriam surgido pela primeira vez nesse universo em uma publicação chamada “Manual de Robótica, 56ª Edição, 2058 d.C.”.

Essas regras deveriam ser implantadas no mais profundo nível das mentes robóticas de maneira que constituíssem as leis mais básicas nas quais essas inteligências artificiais deveriam se pautar e aparecem em praticamente todas as obras do autor, sendo que seus robôs devem obedecê-las (ou a desobediência acaba gerando problemas que são solucionados em suas narrativas).  São elas:

* Primeira Lei: Um robô não pode ferir um ser humano ou, por inação, permitir que um ser humano sofra algum mal.
* Segunda Lei: Um robô deve obedecer às ordens dadas por seres humanos exceto nos casos em que tais ordens entrem em conflito com a Primeira Lei.
* Terceira Lei: Um robô deve proteger sua própria existência desde que tal proteção não entre em conflito com a Primeira ou a Segunda Lei.

https://www.tecmundo.com.br/ciencia/125150-funcionam-tres-leis-robotica-escritor-isaac-asimov-2017.htm


In [269]:
corpus_en = ["A robot may not injure a human being or, through inaction, allow a human being to come to harm.", 
          "A robot must obey the orders given it by human beings except where such orders would conflict with the First Law.",
         "A robot must protect its own existence as long as such protection does not conflict with the First or Second Law."] 

In [2]:
corpus_pt = ["Um robô não pode ferir um ser humano ou, por inação, permitir que um ser humano sofra algum mal.",
             "Um robô deve obedecer às ordens dadas por seres humanos exceto nos casos em que tais ordens entrem em conflito com a Primeira Lei.",
             "Um robô deve proteger sua própria existência desde que tal proteção não entre em conflito com a Primeira ou a Segunda Lei."]

In [3]:
corpus_pt

['Um robô não pode ferir um ser humano ou, por inação, permitir que um ser humano sofra algum mal.',
 'Um robô deve obedecer às ordens dadas por seres humanos exceto nos casos em que tais ordens entrem em conflito com a Primeira Lei.',
 'Um robô deve proteger sua própria existência desde que tal proteção não entre em conflito com a Primeira ou a Segunda Lei.']

In [271]:
pt_stop_words = {'a', 'à', 'adeus', 'agora', 'aí', 'ainda', 'além', 'algo', 'alguém', 'algum', 'alguma', 'algumas', 'alguns', 'ali', 'ampla', 'amplas', 'amplo', 'amplos', 'ano', 'anos', 'ante', 'antes', 'ao', 'aos', 'apenas', 'apoio', 'após', 'aquela', 'aquelas', 'aquele', 'aqueles', 'aqui', 'aquilo', 'área', 'as', 'às', 'assim', 'até', 'atrás', 'através', 'baixo', 'bastante', 'bem', 'boa', 'boas', 'bom', 'bons', 'breve', 'cá', 'cada', 'catorze', 'cedo', 'cento', 'certamente', 'certeza', 'cima', 'cinco', 'coisa', 'coisas', 'com', 'como', 'conselho', 'contra', 'contudo', 'custa', 'da', 'dá', 'dão', 'daquela', 'daquelas', 'daquele', 'daqueles', 'dar', 'das', 'de', 'debaixo', 'dela', 'delas', 'dele', 'deles', 'demais', 'dentro', 'depois', 'desde', 'dessa', 'dessas', 'desse', 'desses', 'desta', 'destas', 'deste', 'destes', 'deve', 'devem', 'devendo', 'dever', 'deverá', 'deverão', 'deveria', 'deveriam', 'devia', 'deviam', 'dez', 'dezanove', 'dezasseis', 'dezassete', 'dezoito', 'dia', 'diante', 'disse', 'disso', 'disto', 'dito', 'diz', 'dizem', 'dizer', 'do', 'dois', 'dos', 'doze', 'duas', 'dúvida', 'e', 'é', 'ela', 'elas', 'ele', 'eles', 'em', 'embora', 'enquanto', 'entre', 'era', 'eram', 'éramos', 'és', 'essa', 'essas', 'esse', 'esses', 'esta', 'está', 'estamos', 'estão', 'estar', 'estas', 'estás', 'estava', 'estavam', 'estávamos', 'este', 'esteja', 'estejam', 'estejamos', 'estes', 'esteve', 'estive', 'estivemos', 'estiver', 'estivera', 'estiveram', 'estivéramos', 'estiverem', 'estivermos', 'estivesse', 'estivessem', 'estivéssemos', 'estiveste', 'estivestes', 'estou', 'etc', 'eu', 'exemplo', 'faço', 'falta', 'favor', 'faz', 'fazeis', 'fazem', 'fazemos', 'fazendo', 'fazer', 'fazes', 'feita', 'feitas', 'feito', 'feitos', 'fez', 'fim', 'final', 'foi', 'fomos', 'for', 'fora', 'foram', 'fôramos', 'forem', 'forma', 'formos', 'fosse', 'fossem', 'fôssemos', 'foste', 'fostes', 'fui', 'geral', 'grande', 'grandes', 'grupo', 'há', 'haja', 'hajam', 'hajamos', 'hão', 'havemos', 'havia', 'hei', 'hoje', 'hora', 'horas', 'houve', 'houvemos', 'houver', 'houvera', 'houverá', 'houveram', 'houvéramos', 'houverão', 'houverei', 'houverem', 'houveremos', 'houveria', 'houveriam', 'houveríamos', 'houvermos', 'houvesse', 'houvessem', 'houvéssemos', 'isso', 'isto', 'já', 'la', 'lá', 'lado', 'lhe', 'lhes', 'lo', 'local', 'logo', 'longe', 'lugar', 'maior', 'maioria', 'mais', 'mal', 'mas', 'máximo', 'me', 'meio', 'menor', 'menos', 'mês', 'meses', 'mesma', 'mesmas', 'mesmo', 'mesmos', 'meu', 'meus', 'mil', 'minha', 'minhas', 'momento', 'muita', 'muitas', 'muito', 'muitos', 'na', 'nada', 'não', 'naquela', 'naquelas', 'naquele', 'naqueles', 'nas', 'nem', 'nenhum', 'nenhuma', 'nessa', 'nessas', 'nesse', 'nesses', 'nesta', 'nestas', 'neste', 'nestes', 'ninguém', 'nível', 'no', 'noite', 'nome', 'nos', 'nós', 'nossa', 'nossas', 'nosso', 'nossos', 'nova', 'novas', 'nove', 'novo', 'novos', 'num', 'numa', 'número', 'nunca', 'o', 'obra', 'obrigada', 'obrigado', 'oitava', 'oitavo', 'oito', 'onde', 'ontem', 'onze', 'os', 'ou', 'outra', 'outras', 'outro', 'outros', 'para', 'parece', 'parte', 'partir', 'paucas', 'pela', 'pelas', 'pelo', 'pelos', 'pequena', 'pequenas', 'pequeno', 'pequenos', 'per', 'perante', 'perto', 'pode', 'pude', 'pôde', 'podem', 'podendo', 'poder', 'poderia', 'poderiam', 'podia', 'podiam', 'põe', 'põem', 'pois', 'ponto', 'pontos', 'por', 'porém', 'porque', 'porquê', 'posição', 'possível', 'possivelmente', 'posso', 'pouca', 'poucas', 'pouco', 'poucos', 'primeira', 'primeiras', 'primeiro', 'primeiros', 'própria', 'próprias', 'próprio', 'próprios', 'próxima', 'próximas', 'próximo', 'próximos', 'pude', 'puderam', 'quais', 'quáis', 'qual', 'quando', 'quanto', 'quantos', 'quarta', 'quarto', 'quatro', 'que', 'quê', 'quem', 'quer', 'quereis', 'querem', 'queremas', 'queres', 'quero', 'questão', 'quinta', 'quinto', 'quinze', 'relação', 'sabe', 'sabem', 'são', 'se', 'segunda', 'segundo', 'sei', 'seis', 'seja', 'sejam', 'sejamos', 'sem', 'sempre', 'sendo', 'ser', 'será', 'serão', 'serei', 'seremos', 'seria', 'seriam', 'seríamos', 'sete', 'sétima', 'sétimo', 'seu', 'seus', 'sexta', 'sexto', 'si', 'sido', 'sim', 'sistema', 'só', 'sob', 'sobre', 'sois', 'somos', 'sou', 'sua', 'suas', 'tal', 'talvez', 'também', 'tampouco', 'tanta', 'tantas', 'tanto', 'tão', 'tarde', 'te', 'tem', 'tém', 'têm', 'temos', 'tendes', 'tendo', 'tenha', 'tenham', 'tenhamos', 'tenho', 'tens', 'ter', 'terá', 'terão', 'terceira', 'terceiro', 'terei', 'teremos', 'teria', 'teriam', 'teríamos', 'teu', 'teus', 'teve', 'ti', 'tido', 'tinha', 'tinham', 'tínhamos', 'tive', 'tivemos', 'tiver', 'tivera', 'tiveram', 'tivéramos', 'tiverem', 'tivermos', 'tivesse', 'tivessem', 'tivéssemos', 'tiveste', 'tivestes', 'toda', 'todas', 'todavia', 'todo', 'todos', 'trabalho', 'três', 'treze', 'tu', 'tua', 'tuas', 'tudo', 'última', 'últimas', 'último', 'últimos', 'um', 'uma', 'umas', 'uns', 'vai', 'vais', 'vão', 'vários', 'vem', 'vêm', 'vendo', 'vens', 'ver', 'vez', 'vezes', 'viagem', 'vindo', 'vinte', 'vir', 'você', 'vocês', 'vos', 'vós', 'vossa', 'vossas', 'vosso', 'vossos', 'zero', '1', '2', '3', '4', '5', '6', '7', '8', '9', '0', '_'}

## Tratamento dos Dados

In [281]:
def stemm_text(text, language):
    stemmer = SnowballStemmer(language) # derivacao
    #remover pontuacao
    remove_punctuation_map = dict((ord(char), None) for char in string.punctuation)
    
    tokens = [word_tokenize(law.lower().translate(remove_punctuation_map)) for law in text] 
    stemmed_tokens = [[stemmer.stem(word) for word in token] for token in tokens]

    corpus = [ ' '.join(word) for word in stemmed_tokens]
    
    return corpus

## Distancia Euclidiana

A distância euclidiana, batizada em homenagem ao sistema geométrico atribuído ao matemático grego Euclides, permitirá medir uma linha reta. Ela é definida como a soma da raiz quadrada da diferença entre x e y em suas respectivas dimensões.

√((x1 – x2)² + (y1 – y2)²)

In [298]:
from scipy.spatial.distance import pdist, squareform
from sklearn.feature_extraction.text import CountVectorizer

In [306]:
def euclid_dist(corpus):
    vectorizer = CountVectorizer(corpus)
    wordcounts = vectorizer.fit_transform(corpus).toarray()
    euclidean_distances = squareform(pdist(wordcounts))
    
    return pd.DataFrame(euclidean_distances, index=corpus, columns=corpus)

In [307]:
corpus_norm = stemm_text(corpus_pt, "portuguese")

In [308]:
dist = euclid_dist(corpus_norm)
dist

C:\Anaconda3\lib\site-packages\sklearn\utils\validation.py:72: FutureWarning: Pass input=['um robô nã pod fer um ser human ou por inaçã permit que um ser human sofr algum mal', 'um robô dev obedec às ordens dad por ser human excet nos cas em que tais ordens entrem em conflit com a primeir lei', 'um robô dev proteg sua própr existent desd que tal proteçã nã entre em conflit com a primeir ou a segund lei'] as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)


,um robô nã pod fer um ser human ou por inaçã permit que um ser human sofr algum mal,um robô dev obedec às ordens dad por ser human excet nos cas em que tais ordens entrem em conflit com a primeir lei,um robô dev proteg sua própr existent desd que tal proteçã nã entre em conflit com a primeir ou a segund lei
um robô nã pod fer um ser human ou por inaçã permit que um ser human sofr algum mal,0.00000,6.000000,5.916080
um robô dev obedec às ordens dad por ser human excet nos cas em que tais ordens entrem em conflit com a primeir lei,6.00000,0.000000,5.196152
um robô dev proteg sua própr existent desd que tal proteçã nã entre em conflit com a primeir ou a segund lei,5.91608,5.196152,0.000000


## Similaridade de Jaccard

O indice de Jaccard aponta a proporção de espécies compartilhadas entre as amostras em relação ao número total de espécies e é dado pela fórmula:

In [141]:
Image(url="img/jaccard.png", width=200, height=100)

* onde “a” é o número de espécies encontrados em ambos os locais (A e B); 
* “b” é o número total de espécies no local B, mas não em A; 
* e “c” é o número de espécies no local A, mas não em B.

In [103]:
from nltk.stem import WordNetLemmatizer
from nltk.stem.snowball import SnowballStemmer
from nltk import word_tokenize

In [311]:
def jaccard_sim(str1, str2): 
    a = set(str1.split()) 
    b = set(str2.split())
    c = a.intersection(b)
    
    jaccard = float(len(c)) / (len(a) + len(b) - len(c)) 
    return jaccard

In [312]:
corpus_norm = stemm_text(corpus_pt, "portuguese")

In [313]:
print("Similaridade Lei 1 x Lei 2 {}".format(jaccard_sim(corpus_norm[0], corpus_norm[1])))
print("Similaridade Lei 1 x Lei 3 {}".format(jaccard_sim(corpus_norm[0], corpus_norm[2])))
print("Similaridade Lei 2 x Lei 3 {}".format(jaccard_sim(corpus_norm[1], corpus_norm[2])))

Similaridade Lei 1 x Lei 2 0.1935483870967742
Similaridade Lei 1 x Lei 3 0.16129032258064516
Similaridade Lei 2 x Lei 3 0.30303030303030304


## Similaridade de Cosseno

A similaridade de cosseno mede o cosseno do ângulo entre dois vetores. 
Isso é calculado como:

In [146]:
Image(url="img/cosseno.png", width=400, height=150)

Com a similaridade de cosseno, precisamos converter sentenças em vetores. 

Uma maneira de fazer isso é usar um conjunto de palavras como TF (frequência do termo) ou TF-IDF (frequência do termo - frequência inversa do documento).

A escolha de TF ou TF-IDF depende da aplicação e é irrelevante para como a similaridade de cosseno é realmente realizada - pois só precisa de vetores. 

TF é bom para similaridade de texto em geral, mas TF-IDF é bom para relevância de consulta de pesquisa.

Outra maneira de calcular a similaridade é usar o Word2Vec ou nossos outros word embeddings personalizados para converter palavras em vetores.

Existem duas diferenças principais entre tf / tf-idf com Word embedding :
* tf / tf-idf cria um número por palavra, Word embedding normalmente cria um vetor por palavra.
2. tf / tf-idf é bom para documentos de classificação como um todo, mas Word embedding são bons para identificar conteúdo contextual.

In [115]:
from sklearn.feature_extraction.text import TfidfVectorizer,ENGLISH_STOP_WORDS

In [262]:
def cosine_sim(corpus, stop_words):
    vect = TfidfVectorizer(stop_words=stop_words).fit(corpus_norm)
    tfidf = vect.transform(corpus_norm)
    
    # vetor com pesos de cada palavra
    txt_df = pd.DataFrame(tfidf.toarray(), columns=vect.get_feature_names()) 
    
    # calcular a similaridade
    pairwise_similarity = tfidf * tfidf.T 
    pairwise_similarity = pd.DataFrame(pairwise_similarity.toarray(), columns=corpus, index=corpus) 
    
    return txt_df, pairwise_similarity
    

In [263]:
corpus_norm = stemm_text(corpus_pt, "portuguese")

In [268]:
corpus_norm

'um robô nã pod fer um ser human ou por inaçã permit que um ser human sofr algum mal'

In [265]:
tfidf_vect, pairwise_similarity = cosine_sim(corpus_norm, pt_stop_words)

In [266]:
tfidf_vect

,cas,conflit,dad,desd,dev,entrem,excet,existent,fer,human,...,permit,pod,primeir,proteg,proteçã,própr,robô,segund,sofr,tais
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.348349,0.529857,...,0.348349,0.348349,0.000000,0.000000,0.000000,0.000000,0.205741,0.000000,0.348349,0.000000
1,0.274816,0.209005,0.274816,0.000000,0.209005,0.274816,0.274816,0.000000,0.000000,0.209005,...,0.000000,0.000000,0.209005,0.000000,0.000000,0.000000,0.162311,0.000000,0.000000,0.274816
2,0.000000,0.250183,0.000000,0.328961,0.250183,0.000000,0.000000,0.328961,0.000000,0.000000,...,0.000000,0.000000,0.250183,0.328961,0.328961,0.328961,0.194290,0.328961,0.000000,0.000000


In [267]:
pairwise_similarity

,um robô nã pod fer um ser human ou por inaçã permit que um ser human sofr algum mal,um robô dev obedec às ordens dad por ser human excet nos cas em que tais ordens entrem em conflit com a primeir lei,um robô dev proteg sua própr existent desd que tal proteçã nã entre em conflit com a primeir ou a segund lei
um robô nã pod fer um ser human ou por inaçã permit que um ser human sofr algum mal,1.000000,0.144137,0.106254
um robô dev obedec às ordens dad por ser human excet nos cas em que tais ordens entrem em conflit com a primeir lei,0.144137,1.000000,0.240694
um robô dev proteg sua própr existent desd que tal proteçã nã entre em conflit com a primeir ou a segund lei,0.106254,0.240694,1.000000


https://medium.com/@raivitor/agrupando-frases-usando-similaridade-por-cosseno-c9d7a55be95b

## Distancia de Levenshtein

FuzzyWuzzy é uma biblioteca de Python usada para entender a similaridade entre textos. Esta analise difusas é definida pelo o processo de localização de strings que correspondem a um determinado padrão. Basicamente, ele usa a distância de Levenshtein para calcular as diferenças entre as sequências.

A distância Levenshtein ou distância de edição entre duas "strings" (duas sequências de caracteres) é dada pelo número mínimo de operações necessárias para transformar uma string no outra. 

Entendemos por "operações" a inserção, deleção ou substituição de um carácter. O nome advém do cientista russo Vladimir Levenshtein, que considerou esta distância já em 1965. É muito útil para aplicações que precisam determinar quão semelhantes dois strings são, como é por exemplo o caso com os verificadores ortográficos.


Por exemplo, a distância Levenshtein entre as palavras inglesas "kitten" (gato) e "sitting" (sentando-se) é 3, já que com apenas 3 edições conseguimos transformar uma palavra na outra, e não há maneira de o fazer com menos de três edições:

* kitten
* sitten (substituição de 'k' por 's')
* sittin (substituição de 'e' por 'i')
* sitting (inserção de 'g' no final)

A distância de Levenshtein pode ser considerada como uma generalização da Distância de Hamming, usada para strings com o mesmo tamanho, a qual só considera edições por substituição. Há também outras generalizações da distância Levenshtein que consideram, por exemplo, a troca de dois caracteres como uma aplicação.


In [84]:
from fuzzywuzzy import fuzz, process

In [282]:
corpus_norm = stemm_text(corpus_pt, "portuguese")

In [293]:
# ratio
print ("\n ratio \n")
print("Similaridade Lei 1 x Lei 2 {}".format(fuzz.ratio(corpus_norm[0], corpus_norm[1])))
print("Similaridade Lei 1 x Lei 3 {}".format(fuzz.ratio(corpus_norm[0], corpus_norm[2])))
print("Similaridade Lei 2 x Lei 3 {}".format(fuzz.ratio(corpus_norm[1], corpus_norm[2])))


 ratio 

Similaridade Lei 1 x Lei 2 40
Similaridade Lei 1 x Lei 3 30
Similaridade Lei 2 x Lei 3 66


**Razão Parcial**

FuzzyWuzzy também tem funções mais poderosas para ajudar na correspondência de strings em situações mais complexas. A função `fuzz.partial_ratio()` nos permite realizar a correspondência de substring. Isso funciona pegando a string mais curta e combinando-a com todas as substrings do mesmo comprimento.

In [292]:
#partial_ratio
print ("\n partial_ratio \n")
print("Similaridade Lei 1 x Lei 2 {}".format(fuzz.partial_ratio(corpus_norm[0], corpus_norm[1])))
print("Similaridade Lei 1 x Lei 3 {}".format(fuzz.partial_ratio(corpus_norm[0], corpus_norm[2])))
print("Similaridade Lei 2 x Lei 3 {}".format(fuzz.partial_ratio(corpus_norm[1], corpus_norm[2])))


 partial_ratio 

Similaridade Lei 1 x Lei 2 46
Similaridade Lei 1 x Lei 3 33
Similaridade Lei 2 x Lei 3 59


**Razão de classificação de token**

FuzzyWuzzy também tem funções de token que tokenizar as strings, mudar as maiúsculas para minúsculas e remover a pontuação. A `token_sort_ratio()` função classifica as strings em ordem alfabética e depois as junta. Então, o `fuzz.ratio()` é calculado. Isso pode ser útil quando as strings que você está comparando têm a mesma grafia, mas não estão na mesma ordem.

In [291]:
#token_sort_ratio
#partial_ratio
print ("\n token_sort_ratio \n")
print("Similaridade Lei 1 x Lei 2 {}".format(fuzz.token_sort_ratio(corpus_norm[0], corpus_norm[1])))
print("Similaridade Lei 1 x Lei 3 {}".format(fuzz.token_sort_ratio(corpus_norm[0], corpus_norm[2])))
print("Similaridade Lei 2 x Lei 3 {}".format(fuzz.token_sort_ratio(corpus_norm[1], corpus_norm[2])))



 token_sort_ratio 

Similaridade Lei 1 x Lei 2 48
Similaridade Lei 1 x Lei 3 47
Similaridade Lei 2 x Lei 3 65


A  função `token_set_ratio()` é semelhante à `token_sort_ratio()`, exceto que tira os tokens comuns antes de calcular o fuzz.ratio()entre as novas strings. Essa função é mais útil quando aplicada a um conjunto de strings com uma diferença significativa de comprimento.

In [294]:
print ("\n token_set_ratio \n")
print("Similaridade Lei 1 x Lei 2 {}".format(fuzz.token_set_ratio(corpus_norm[0], corpus_norm[1])))
print("Similaridade Lei 1 x Lei 3 {}".format(fuzz.token_set_ratio(corpus_norm[0], corpus_norm[2])))
print("Similaridade Lei 2 x Lei 3 {}".format(fuzz.token_set_ratio(corpus_norm[1], corpus_norm[2])))


 token_set_ratio 

Similaridade Lei 1 x Lei 2 55
Similaridade Lei 1 x Lei 3 41
Similaridade Lei 2 x Lei 3 71


**Extract**

O FuzzyWuzzy também vem com um módulo útil, processo, que retorna as strings junto com uma pontuação de similaridade de um vetor de strings. Tudo que você precisa fazer é chamar a função `extract()` após o processo.

In [295]:
process.extract("maldade", corpus_norm, limit=3)

[('um robô nã pod fer um ser human ou por inaçã permit que um ser human sofr algum mal',
  36),
 ('um robô dev obedec às ordens dad por ser human excet nos cas em que tais ordens entrem em conflit com a primeir lei',
  32),
 ('um robô dev proteg sua própr existent desd que tal proteçã nã entre em conflit com a primeir ou a segund lei',
  18)]

https://ichi.pro/pt/correspondencia-de-string-com-fuzzywuzzy-256747878976132

https://pypi.org/project/fuzzywuzzy/

https://github.com/ztane/python-Levenshtein/


## Word embedding